# Links

This file creates a class 'Links', whose objects contain the essential parameters for a directionaal link like : nodes(source, destination pair), weight, availablity of various wavelengths, etc.

Furthermore, there are various methodsto initialize, update, displaying links. Also, methods to calculate Path resources, TUR, and FF approach are created.

In [6]:
# Need to make a list of k shortest paths for each node pair. Will save time.

class Links:
    """
    Represents a network of links connecting nodes. Each link has specific resources (time slots)
    allocated for different security levels (SL) and traditional data channels (TDC).
    
    Attributes:
        channel_ts (dict): A dictionary mapping security level ("tdc", "high", "medium", "low") to the
                            corresponding total number of time slots in the channel.
        n_ts (dict): A dictionary mapping security level ("tdc", "high", "medium", "low") to the
                     number of time slots in the corresponding channel.
        channel (dict): A dictionary mapping security level ("tdc", "high", "medium", "low") to the
                         channel index.
        priority (dict): A dictionary mapping channel index (0-3) to the corresponding security level.
        link_ts (int): The total number of time slots in each link (sum of all security level time slots).
        links (np.ndarray, optional): A 2D NumPy array containing Link objects at corresponding node indices
                                       (initialized in `initialize_links`).
        indices (np.ndarray, optional): A mask indicating the location of each link in the `links` array
                                        (initialized in `initialize_links`).
        ordered_indices (list, optional): A list of tuples representing ordered pairs of nodes for each link
                                          (initialized in `initialize_links`).
    
    """


    ## Class variables
    
    #links = []
    #indices : a mask
    #unordered_indices = []    # Not useful in case of bidirectional graph
    #numLinks = len(ordered_indices)
    #key_reserve
    #total_ts -> of the entire network
    #link_ts -> of 1 link
    #UTS -> utilised time slots(Total)
    #ATS -> Available time slots(totsl)

    channel_ts = {1 : 8, 2 : 10, 3 : 12}
    n_ts = {"tdc" : 47, "high" : channel_ts[1], "medium" : channel_ts[2], "low" : channel_ts[3]}    # Creating a dictionary with sl and the corresponding number of time-slots in the channel

    channel = {"tdc" : 0, "high" : 1, "medium" : 2, "low" : 3}    # A dictionary for channel from sl 
    priority = {0 : "tdc", 1 : "high", 2 : "medium", 3 : "low"}    # A dictionary for sl from channel
    
    link_ts = channel_ts[1] + channel_ts[2] + channel_ts[3]    # Total ts in each link
    
    def __init__(self, nodes, weight):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
        self.nodes = nodes
        self.weight = weight
        
        self.lambda_tdc = np.ones(47, dtype = bool)
        
        #self.total_ts = Links.total_ts
        
        self.occupied_ts = np.zeros(4).astype(int)    # Stores the number of occupied time slots [total, q1, q2, q3]
        self.available_ts = np.array([Links.link_ts,    # Stores the number of available time slots [total, q1, q2, q3]
                                     Links.channel_ts[1], Links.channel_ts[2], Links.channel_ts[3]])
        
        self.lambda_q1 = np.ones(Links.channel_ts[1], dtype = bool)    # for high sl
        self.lambda_q2 = np.ones(Links.channel_ts[2], dtype = bool)    # for medium sl    
        self.lambda_q3 = np.ones(Links.channel_ts[3], dtype = bool)    # for low sl
        


    def update_link(self, channel, slot):
        """
        Updates the availability of a time slot in a specific channel.

        Args:
            channel (int): The channel index (0 for TDC, 1-3 for security levels).
            slot (int): The time slot index.
        """
        #QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
        QSC = [1, 2, 3]
        TDC = [0]
        AC = QSC + TDC
        #if channel not in AC or ts < 0:
            #print("Invalid Update request!")

        if channel == 0:
            self.lambda_tdc[slot] = False
        
        elif channel == 1 and slot < len(self.lambda_q1):    #self.channel_ts(channel)
            self.lambda_q1[slot] = False 
            
        elif channel == 2 and slot < len(self.lambda_q2):
            self.lambda_q2[slot] = False
            
        elif channel == 3 and slot < len(self.lambda_q3):
            self.lambda_q3[slot] = False

        else:
            raise ValueError("Invalid time slot value")

        if channel != 0:
            self.occupied_ts[channel] += 1 
            self.occupied_ts[0] += 1

            self.available_ts[channel] -= 1 
            self.available_ts[0] -= 1
        
    
    def display_info(self, wl_info = False):
        """
        Displays information about the link, including available and occupied time slots.

        Args:
            wl_info (bool, optional): If True, also displays detailed wavelength information. Defaults to False.
        """
        q1_count = self.available_ts[1] # nonzero => Available slots
        q2_count = self.available_ts[2] #== True
        q3_count = self.available_ts[3] #== True
        
        tdc_count = np.count_nonzero(self.lambda_tdc) #== True
        print(f"Link {self.nodes} : lambda_tdc_count = {tdc_count}, lambda_q1_count = {q1_count}, lambda_q2_count = {q2_count}, lambda_q3_count = {q3_count}, occupied_ts = {self.occupied_ts}, available_ts = {self.available_ts}")

        if wl_info:    # To show the wavelength occupancy
            print(f"QSC: \n q1 : {self.lambda_q1}, q2 : {self.lambda_q2}, q3 : {self.lambda_q3}, tdc : {self.lambda_tdc}")
        
  ###################################################################################################################################################
    
    @classmethod
    def initialize_links(cls, edges):    # Initializing all the links and the individual link resources
        """
        Initializes all links in the network based on the provided edges data.

        Args:
            edges (list): A list of tuples representing edges in the graph (source, destination, weight).

        Returns:
            tuple: A tuple containing the `links`, `indices`, and `ordered_indices` arrays.
        """
        cls.total_ts = cls.link_ts*len(edges)
        cls.UTS = 0
        cls.ATS = cls.total_ts
        
        num_Nodes = max(set([nodes for row in edges for nodes in row[:2]]))    # A list of all the values/nodes in first 2 columns : s & d
        
        cls.links = np.zeros([num_Nodes+1, num_Nodes+1], dtype = object)    # Matrix containing link object at position s, d. Will have redundant entries.
        cls.ordered_indices = []

        for (s, d, w) in edges:
            nodes = (s, d)
            link = cls(nodes, w)
            cls.links[s, d] = link   
            cls.ordered_indices.append(nodes)
        # In the above call of __init__ constructor, the wavelength resources have also been initialized to all available(True)
                    
        cls.indices = cls.links != 0    # A mask that holds the location of each link. NOTE : It stores ordered pair
        
        # return links, indices, ordered_indices
        

    @classmethod
    def path_resources(cls, path, sl):    # To check for the available time slots, following the continuity constraint
        """
        Checks for available time slots along a specified path for a given security level.

        Args:
            path (list): A list of nodes representing the path.
            sl (str): The security level.

        Returns:
            tuple: A tuple containing lists of available time slots for TDC and the security level.
        """
        available_tdcs = [True] * cls.n_ts["tdc"]    # For traditional data channel slots
        available_ts = [True] * cls.n_ts[sl]    # Creating a base boolean array of the size corresponding to the specific CR's sl

        for s, d in zip(path, path[1:]):    # Taking consecutive pairs of nodes and selecting the particular channel
            if sl == "high":
                band = cls.links[s, d].lambda_q1
            elif sl == "medium":
                band = cls.links[s, d].lambda_q2
            else:
                band = cls.links[s, d].lambda_q3

            # Checking for continuity constraints in ts of quantum channel
            available_ts = [a and b for a, b in zip(available_ts, band)]
            available_tdcs = [a and b for a, b in zip(available_tdcs, cls.links[s, d].lambda_tdc)]
            
        return available_ts, available_tdcs


    @classmethod
    def ASLC(cls, ch, path, aslc, beta_1 = 1, beta_2 = 0.5):
        """
        Implements the Adaptive Spectrum Leasing Channel (ASLC) algorithm.

        Args:
            ch (int): The channel index.
            path (list): The path to be considered.
            aslc (str): The ASLC strategy (ASSL or AWSL).
            beta_1 (float, optional): The first threshold parameter for AWSL. Defaults to 1.
            beta_2 (float, optional): The second threshold parameter for ASSL. Defaults to 0.5.

        Returns:
            str: The selected security level.
        """
        
        NT = []    # equivalent to dict channel_ts. starts from index 0 : for high priority
        OT = []    # Number of ts occupied along the ENTIRE path
        for i in range(1, 4):
            nt = cls.channel_ts[i]    # Total time slot in channel/n-th wavelength
            NT.append(nt)
                
            available_ts, available_tdcs = cls.path_resources(path, cls.priority[i])    # Available ts in the path for a particular wavelength
            ot = nt - np.count_nonzero(available_ts)    # Denotes the number of time-slot continuity occupied along the path for the given sl
            OT.append(ot)
                
        # ch could be 1, 2 or 3
        # n = ch - 1    # n could be 0, 1 or 2    # no need for n

        if aslc == "ASSL" and ch != 1:    # ch = 2 or 3. n = 1 or 2
            if OT[ch-1] >= beta_2 * NT[ch-1]:    # If the occupied resources in the higher priority are greater than a threshold, don't allocate to it
                QW = cls.priority[ch]
            else:
                QW = cls.priority[ch-1]

        elif aslc == "AWSL" and ch != 3:    # ch = 1 or 2. n = 0 or 1
            if OT[ch] >= beta_1 * NT[ch]:    # If the resources in current priority are more than a threshold, allocate to a lower priority
                QW = cls.priority[ch+1]      # NOTE : Beta_2 = 1... so it will only use this when the current level is full
            else:
                QW = cls.priority[ch]

        else:    # For SSL and cases in ASSL but high priority, or AWSL but low priority
            QW = cls.priority[ch]
         
        return QW
            
        
    @classmethod
    def FF(cls, path, cr, aslc = "ssl"):
        """
        Performs First Fit (FF) spectrum allocation for a Connection Request (CR).

        Args:
            path (list): The path for the CR.
            cr (CR): The Connection Request object.
            aslc (str, optional): The ASLC strategy to use. Defaults to "ssl".

        Returns:
            bool: True if allocation is successful, False otherwise.
        """
        # First check for availability of resources
        
        cr.sl = cls.ASLC(cls.channel[cr.sl], path, aslc)
        sl = cr.sl
        
        available_ts, available_tdcs = cls.path_resources(path, sl)    # List of available time slots in the channel along the entire path
         
        if np.count_nonzero(available_ts) == 0 or np.count_nonzero(available_tdcs) == 0:    # A pre-condition to check if available slots are present
            cr.update_status("blocked")
            return False

        # Some issue here : try ignoring the condition on tdc path
        qscs = np.nonzero(available_ts)[0][0]    # This function will give the index of 1st non-zero/True element in the list
        tdcs = np.nonzero(available_tdcs)[0][0]    # At present, assuming only one tdcs is needed
        
        for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
            link = cls.links[s, d]

            link.update_link(cls.channel["tdc"], tdcs)
            link.update_link(cls.channel[sl], qscs)    # Updating the i_th ts in channel for sl to False
            
            cls.UTS += 1
            cls.ATS -+ 1

            CR.Nsp += cls.n_ts[sl]
            
        allocated_resources = [cls.channel[sl], qscs]
        cr.update_status("allocated", allocated_resources, path)
        CR.Allocated += 1
        
        return True
        
        
    @classmethod
    def TUR(cls, channel = 'total'):    # Returns the total TUR, if no 2nd argument given    
        """
        Calculates the Timeslot Utilization Ratio (TUR) for the network.
    
        Args:
            channel (str, optional): Specifies whether to calculate TUR for all channels or a specific channel.
                - 'total' (default): Calculates the overall TUR.
                - Other values: Calculates TUR for the specified channel (not implemented).
    
        Returns:
            float or list:
                - If `channel` is 'total', returns the overall TUR.
                - Otherwise, returns a list of TUR values for each channel.
        """
        # Only for quantum channel
        util_ts = np.zeros(4)
        tur = util_ts
        num_links = len(cls.ordered_indices)    # This also takes in consideration the symmetric part of a unidirectional link
        
        for nodes in cls.ordered_indices :
            link = cls.links[nodes]
            util_ts += link.occupied_ts

        tur[0] = util_ts[0]/cls.total_ts
        for i in range(1, 4):
            tur[i] = util_ts[i]/(len(cls.ordered_indices)*cls.channel_ts[i])
        
        print("The time-slot utilization ratio(TUR) is : ", tur)
        
        if channel == 'all': return tur
        else: return tur[0]       
            
    
    @classmethod
    def display_all_links(cls, wl_info = False):
        for nodes in cls.ordered_indices :
            cls.links[nodes].display_info(wl_info)


In [1]:
# Example of NSFNET 